In [2]:
import spacy
import yaml
import re

In [3]:
from tweepy import OAuthHandler
from tweepy.api import API as Twitter
from tweepy.error import TweepError
from tweepy.parsers import JSONParser

In [99]:
nlp = spacy.load("en_core_web_sm")

In [100]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x1b7bb8f1710>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x1b7bcfbcb28>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x1b7bcfbcb88>)]

In [3]:
stream = open('C:/Users/jjc/Misc/twitter_api_details.yml', "r")
conf = yaml.load(stream)

C:\Users\jjc\Anaconda3\envs\nlp-workshop\lib\site-packages\ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [4]:
conf

{'twitter': {'api': {'pause': 1,
   'app': {'key': 'X9JqOAFJ8GjYNqvEolEpvZdE2',
    'secret': '4j5TdZC2o5YfSFnlASa10x9KLE3FN4P5FTOwHLSrx8FYYem7c1',
    'token': '2620986611-pOP0uh0cEkgh8yxmJdNEpK3kD4DIKTSJvmKJxR6',
    'token_secret': 'yslRiqrk6EriZxFmeWAkJItWOD9PtdVOLe3h5IaoskRVY'}}}}

In [8]:
auth = OAuthHandler(conf["twitter"]["api"]["app"]["key"], conf["twitter"]["api"]["app"]["secret"])
auth.set_access_token(conf["twitter"]["api"]["app"]["token"], conf["twitter"]["api"]["app"]["token_secret"])
twitter = Twitter(auth, parser=JSONParser())

In [15]:
results = twitter.search('#GoT', lang='en')

In [17]:
len(results['statuses'])

15

In [72]:
#http_regexp = 'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'
http_regexp = 'https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}A'

In [73]:
re.findall(http_regexp,'I miss the old Tyrion #GoT #GameofThrones https://t.co/YbBI6vkzYh https://t.co/j6XbRWYPqN' )

['https://t.co/YbBI6vkzYh', 'https://t.co/j6XbRWYPqN']

In [75]:
pattern = re.compile(http_regexp)

In [76]:
pattern.sub('','I miss the old Tyrion #GoT #GameofThrones https://t.co/YbBI6vkzYh https://t.co/j6XbRWYPqN')

'I miss the old Tyrion #GoT #GameofThrones  '

In [94]:
def remove_urls(text):
    
    return pattern.sub('',text)

def remove_hashes(text):
    
    return text.replace('#','')

def remove_newline(text):
    return text.replace('\n',' ')

In [95]:
def clean_tweets(tweets):
    out_list=[]

    for tweet in tweets['statuses']:
        text= tweet['text']
        url_clean = remove_urls(text)
        hash_clean = remove_hashes(url_clean)
        newline_clean = remove_newline(hash_clean)
        out_list.append(newline_clean)
        
    return out_list

In [96]:
cleaned_texts = clean_tweets(results)

In [97]:
cleaned_texts

['RT @SDinwiddie_25: Q: Favorite Game of Thrones scene AskSpencer - @DanParzych  A: GOT GameOfThrones K8IROS ',
 'RT @dreadnstyn: If You Have 400k Less Followers  Follow me @dreadnstyn  100 Rts For 100 Followers🇳🇬  200 Rts For 300 Followers🇳🇬  300 Rts F…',
 'RT @dreadnstyn: If You Have 80k Less Followers  Follow me @dreadnstyn  500 Rts For 500 Followers🇳🇬  600 Rts For 600 Followers🇳🇬  700 Rts Fo…',
 'RT @dreadnstyn: If You Have 400k Less Followers  Follow me @dreadnstyn  100 Rts For 100 Followers🇳🇬  200 Rts For 300 Followers🇳🇬  300 Rts F…',
 'RT @dreadnstyn: If You Have 400k Less Followers  Follow me @dreadnstyn  100 Rts For 100 Followers🇳🇬  200 Rts For 300 Followers🇳🇬  300 Rts F…',
 'RT @dreadnstyn: If You Have 400k Less Followers  Follow me @dreadnstyn  100 Rts For 100 Followers🇳🇬  200 Rts For 300 Followers🇳🇬  300 Rts F…',
 'Repost @markmannphoto with make_repost ・・・ @maisie_williams was pretty suprised watching got last week..… ',
 'RT @dreadnstyn: If You Have 400k Less Followers  F

In [102]:
for sentence in cleaned_texts:
    doc = nlp(sentence)
    entities = doc.ents
    if len(entities) > 0:
        output = dict(
            sentence=sentence,
            entities=entities
        )
        print(output)

{'sentence': 'RT @dreadnstyn: If You Have 400k Less Followers  Follow me @dreadnstyn  100 Rts For 100 Followers🇳🇬  200 Rts For 300 Followers🇳🇬  300 Rts F…', 'entities': (🇬  200, Rts For, 300, 🇬  , Rts F)}
{'sentence': 'RT @dreadnstyn: If You Have 80k Less Followers  Follow me @dreadnstyn  500 Rts For 500 Followers🇳🇬  600 Rts For 600 Followers🇳🇬  700 Rts Fo…', 'entities': (600, 🇬  , Rts Fo)}
{'sentence': 'RT @dreadnstyn: If You Have 400k Less Followers  Follow me @dreadnstyn  100 Rts For 100 Followers🇳🇬  200 Rts For 300 Followers🇳🇬  300 Rts F…', 'entities': (🇬  200, Rts For, 300, 🇬  , Rts F)}
{'sentence': 'RT @dreadnstyn: If You Have 400k Less Followers  Follow me @dreadnstyn  100 Rts For 100 Followers🇳🇬  200 Rts For 300 Followers🇳🇬  300 Rts F…', 'entities': (🇬  200, Rts For, 300, 🇬  , Rts F)}
{'sentence': 'RT @dreadnstyn: If You Have 400k Less Followers  Follow me @dreadnstyn  100 Rts For 100 Followers🇳🇬  200 Rts For 300 Followers🇳🇬  300 Rts F…', 'entities': (🇬  200, Rts For, 300, 🇬  ,

### Model

In [6]:
import pandas as pd

In [37]:
from fastai.text import load_data, text_classifier_learner, AWD_LSTM
import torch
from pathlib import Path, PosixPath, PurePosixPath

In [9]:
torch.cuda.get_device_name(0)

'GeForce MX150'

In [38]:
path=PurePosixPath('./model/sentiment')
path

PurePosixPath('model/sentiment')

In [39]:
path/'data_clas.pkl'

PurePosixPath('model/sentiment/data_clas.pkl')

In [40]:
bs=48
#path=Path('c:/Users/jjc/Projects/pydata2019/pydata2019-nlp-system/step3_nlp/model/sentiment/')
data_clas = load_data(path, 'data_clas.pkl', bs=bs)
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)

NotImplementedError: cannot instantiate 'PosixPath' on your system